# **Convergence of Selection to Uniform Populations**
In this notebook I will explore simple evolutionary algorithms using only selection operators (no mutation/crossover).

The expected result for each algorithm is to have a uniform population where all of the individuals are the same.

TODO: selection operators that return lists don't seem to be working with `pipe()`, see Truncation Selection.

In [1]:
from toolz import pipe
from tqdm import tqdm
import matplotlib.pyplot as plt

from leap_ec.individual import Individual
from leap_ec.decoder import IdentityDecoder
from leap_ec.context import context

import leap_ec.ops as ops
from leap_ec.real_rep.problems import SpheroidProblem
from leap_ec.real_rep.initializers import create_real_vector
from leap_ec import util

# **Problem & Representation**
The problem is simple 2-D spheroid problem.

The genome of each individual is a 2 dimensional real vector.

Each type of selection will involve running with the following parameters:
- population_size: 10
- selection only (no mutation/crossover)

In [11]:
def genome_diff(population):
    ''' Computes | max(population) - min(population) | '''
    genomes = [ind.genome[0] for ind in population]
    return abs(max(genomes) - min(genomes))


def run_with_selection(select_op, **kwargs):
    ''' Performs 50,000 runs of selection only evolution '''
    avg_gen = 0
    for i in tqdm(range(50000)):
        # create population
        parents = Individual.create_population(10, initialize=create_real_vector([(-5.12, 5.12), (-5.12, 5.12)]),
                                               decoder=IdentityDecoder(), problem=SpheroidProblem())

        # evaulate current population
        parents = Individual.evaluate_population(parents)

        generation_counter = util.inc_generation(context=context)

        while genome_diff(parents) != 0.0:
            offspring = pipe(parents,
                             lambda x: select_op(x, **kwargs),
                             ops.clone,
                             ops.evaluate,
                             ops.pool(size=len(parents))
                             )
            parents = offspring
            generation_counter()
        avg_gen += ((context['leap']['generation'] - avg_gen)/(i+1))

    print(f'Average generation until uniform: {avg_gen}')

## **Random Selection**
Random selection converges to a uniform generation in ~17.6 generations.

In [12]:
run_with_selection(ops.random_selection)

100%|██████████| 50000/50000 [01:00<00:00, 831.62it/s]

Average generation until uniform: 17.561719999999923


## **Tournament Selection**
Tournament selection converges to a uniform population in ~5.2 generations.

In [16]:
run_with_selection(ops.tournament_selection, k=2)

100%|██████████| 50000/50000 [00:24<00:00, 2000.33it/s]

Average generation until uniform: 5.213119999999976


## **Truncation Selection**
Truncation selection converges to a uniform population in ~ generations.

NOTE: Not working I think because it returns a `List` instead of `Iterator`.

In [17]:
# run_with_selection(ops.truncation_selection, size=5)

## **Cyclic Selection**
Cyclic selection converges to a uniform population in ~ generations.

NOTE: Not working because it is taking too long to run. I guess this one is not converging?

In [20]:
# run_with_selection(ops.naive_cyclic_selection)